# Get own implied volatilities and compare them to Deribits

In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns
import plotly.graph_objects as go
import datetime
from scipy import stats as sps

from utils import get_human_timestamp
import plotly
import plotly.graph_objs as go
from typing import Union
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
df = pd.read_csv(f"./datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
data = df.copy()

In [52]:
#Ньютон-Рафсон
def get_implied_volatility_call(C: Union[float, int], K: Union[float, int], T: Union[float, int], 
                               F: Union[float, int], r: Union[float, int], error: Union[float, int]):
    vol = 1.0
    dv = error + 1
    while abs(dv) > error:
        d1 = (np.log(F / K) + 0.5 * vol ** 2 * T) \
                / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        D = np.exp(-r * T)
        call_price = F * sps.norm.cdf(d1) - K * sps.norm.cdf(d2) * D
        Vega = F * np.sqrt(T / np.pi / 2) * np.exp(-0.5 * d1**2)
        PriceError = call_price - C
        dv = PriceError / Vega
        vol = vol - dv
    return vol

In [58]:
# 69.75
C = 0.0146*1294.5900
K = 1400
T = 0.5
F = 1294.5900
r = 0.0
error = 0.01
get_implied_volatility_call(C = C, K = K, T = T, F = F, r = r, error = error)

0.1493149990165815

In [40]:
data

,exchange,symbol,timestamp,local_timestamp,type,strike_price,expiration,open_interest,last_price,bid_price,bid_amount,bid_iv,ask_price,ask_amount,ask_iv,mark_price,mark_iv,underlying_index,underlying_price,delta,gamma,vega,theta,rho
0,deribit,ETH-9DEC22-1400-C,1669852800002000,1669852800011631,call,1400,1670572800000000,12888.0,0.0145,0.0140,309.0,68.60,0.0150,230.0,70.70,0.0146,69.75,ETH-9DEC22,1294.5900,0.24508,0.00230,0.61504,-2.57385,0.06814
1,deribit,ETH-16DEC22-1300-P,1669852800011000,1669852800014218,put,1300,1671177600000000,185.0,0.0640,0.0630,127.0,74.35,0.0650,214.0,76.80,0.0632,74.51,SYN.ETH-16DEC22,1294.5233,-0.48057,0.00202,1.05725,-2.56887,-0.29568
2,deribit,ETH-16DEC22-1500-C,1669852800010000,1669852800017777,call,1500,1671177600000000,7503.0,0.0150,0.0135,426.0,72.46,0.0145,219.0,74.30,0.0140,73.44,SYN.ETH-16DEC22,1294.5233,0.18313,0.00136,0.70376,-1.68530,0.09196
3,deribit,ETH-30DEC22-1400-P,1669852800013000,1669852800018794,put,1400,1672387200000000,11013.0,0.1275,0.1310,101.0,71.44,0.1335,83.0,73.73,0.1325,72.78,ETH-30DEC22,1294.2700,-0.60928,0.00144,1.40850,-1.74735,-0.77151
4,deribit,ETH-2DEC22-1300-C,1669852800013000,1669852800019006,call,1300,1669968000000000,26858.0,0.0145,0.0140,32.0,66.44,0.0150,85.0,70.60,0.0147,69.18,ETH-2DEC22,1294.4500,0.46756,0.00735,0.31108,-8.07055,0.02142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22492236,deribit,BTC-30DEC22-34000-C,1669939199983000,1669939199999665,call,34000,1672387200000000,3670.1,0.0005,NaN,NaN,NaN,0.0005,0.1,95.08,0.0004,93.19,BTC-30DEC22,16953.1600,0.00538,0.00000,0.72864,-1.19821,0.06533
22492237,deribit,BTC-29SEP23-10000-C,1669939199983000,1669939199999801,call,10000,1695974400000000,0.6,0.4775,0.4650,10.1,70.33,0.4915,10.0,83.81,0.4728,74.39,BTC-29SEP23,16962.0900,0.86858,0.00002,32.84842,-4.05454,55.42993
22492238,deribit,BTC-31MAR23-5000-C,1669939199984000,1669939199999808,call,5000,1680249600000000,2.7,0.7390,0.7005,12.0,NaN,0.7120,12.0,126.49,0.7089,113.66,BTC-31MAR23,16946.2200,0.98621,0.00000,3.41384,-1.62576,15.36320
22492239,deribit,BTC-9DEC22-13500-C,1669939199988000,1669939199999809,call,13500,1670572800000000,0.0,NaN,0.0005,0.3,NaN,NaN,NaN,NaN,0.2059,84.57,BTC-9DEC22,16975.1700,0.97563,0.00003,1.37670,-7.93844,2.62528
